In [117]:
from sage.misc.dev_tools import import_statements

In [118]:
import_statements(round)

from builtins import round


In [1]:
from itertools import combinations_with_replacement

from ehrhart_quasi_polynomial.ehrhart_piecewise import (
    PiecewiseEhrhartQuasiPolynomial as PEQP,
    create_polytope_from_matrix,
    secondary_fan,
    _process_fan_vectors,
    _compute_change_of_basis_matrices,
    _compute_periods,
    _generate_cone_points)

from ehrhart_quasi_polynomial import *

In [2]:
from sage.calculus.var import var
from sage.functions.other import ceil, factorial
from sage.geometry.cone import Cone
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.matrix.constructor import Matrix
from sage.modules.free_module_element import free_module_element
from sage.modules.free_quadratic_module_integer_symmetric import IntegralLattice
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.rational_field import QQ

In [8]:
ehr = lambda A, b: ehrhart_quasi_polynomial(create_polytope_from_matrix(A, b).Vrepresentation())
num_int_points = lambda A, b: len(create_polytope_from_matrix(A, b).integral_points())

In [121]:
def __call__(self, point):
    if len(point) != self._amb_dim:
            raise ValueError("Dimension of ``point`` needs to be equal to the ambient"
                              f" dimension of ``self`` which is {self._amb_dim}.")
    
    for k, cone_dict in enumerate(self._cone_dicts):
            if point in cone_dict["cone"]:
                # print(True)
                off_set = cone_dict["quotient"][cone_dict["quotient"](point)[0]]
                eval_p = ( cone_dict["change_of_basis_inverse"]*(
                    free_module_element(point) - off_set.lift()) )[:self._num_variables]

                result = cone_dict["polynomials"][off_set](*eval_p)
                if isinstance(result, int):
                    return result
                else:
                    return round(result), k, off_set, eval_p
    return 0, None, None, point

PEQP.__call__ = __call__

In [83]:
def test_combinations(p, A, xrange=range(-10, 11)):
        for b in combinations_with_replacement(xrange, A.nrows()):
            expected = num_int_points(A, b)
            actual = p(b)
            if actual[0] != expected:
                print(b, expected, actual)
        print("all points were tested")

In [76]:
A = Matrix([[-1, 0], [0, -1], [1, 1]])
p = PEQP(A)

In [16]:
p._cone_dicts[0]["polynomials"]

{(0): 9/2*x^2 + 9/2*x + 1, (1): 9/2*x^2 + 3/2*x, (2): 9/2*x^2 - 3/2*x}

In [84]:
test_combinations(p, A)

all points were tested


In [85]:
B = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
q = PEQP(B)

In [65]:
# q._cone_dicts[0]["polynomials"]

In [86]:
test_combinations(q, B)

all points were tested


In [124]:
C = Matrix([[-1, 0], [0, -1], [1, 2], [0, 1]])
r = PEQP(C)

In [125]:
test_combinations(r, C, range(-6, 7))

all points were tested


In [108]:
b = free_module_element([0, 0, 0, 1]) + r._rays[2]*2
b, r(b)

((2, 6, 2, 3), (323/4, 0, (0, 3), (11/8, 13/8)))

In [56]:
for off_set, poly in list(r._cone_dicts[0]["polynomials"].items()):
    print(ehr(C, ray + off_set.lift()))

QuasiPolynomial given by 
[1] + [8]*t + [16]*t^2
QuasiPolynomial given by 
[1] + [9]*t + [18]*t^2
QuasiPolynomial given by 
[1] + [10]*t + [16]*t^2
QuasiPolynomial given by 
[1] + [11]*t + [10]*t^2
QuasiPolynomial given by 
[1] + [12]*t
QuasiPolynomial given by 
[0]
QuasiPolynomial given by 
[0]
QuasiPolynomial given by 
[0]
QuasiPolynomial given by 
[1] + [6]*t + [9]*t^2
QuasiPolynomial given by 
[1, 3/4] + [7]*t + [49/4]*t^2
QuasiPolynomial given by 
[1] + [8]*t + [15]*t^2
QuasiPolynomial given by 
[1] + [9]*t + [14]*t^2
QuasiPolynomial given by 
[1] + [10]*t + [9]*t^2
QuasiPolynomial given by 
[1] + [11]*t
QuasiPolynomial given by 
[0]
QuasiPolynomial given by 
[0]
QuasiPolynomial given by 
[1] + [4]*t + [4]*t^2
QuasiPolynomial given by 
[1, 3/4] + [5]*t + [25/4]*t^2
QuasiPolynomial given by 
[1] + [6]*t + [9]*t^2
QuasiPolynomial given by 
[1] + [7]*t + [12]*t^2
QuasiPolynomial given by 
[1] + [8]*t + [12]*t^2
QuasiPolynomial given by 
[1] + [9]*t + [8]*t^2
QuasiPolynomial given by 

In [114]:
r._scalars = (2, 2, 2)
r._cone_dicts = r._generate_cone_dicts()
r._compute_piecewise()

In [123]:
test_combinations(r, C, range(-10, 11))

all points were tested


In [127]:
def _compute_piecewise(self):
        max_degree = self._A.ncols()
        needed_points = factorial(self._num_variables + max_degree + 1)//(
            factorial(self._num_variables)*factorial(max_degree) )

        points = _generate_cone_points(self._num_variables, needed_points)
        for cone_dict in self._cone_dicts:
            cone_points = {0: (points, [tuple(b) for b in points])}
            ray_sum = sum(cone_dict["scaled_rays"])
            print(cone_dict["quotient"])

            polynomials = {}
            for off_set in cone_dict["quotient"]:
                print(off_set)
                nudge = off_set.lift()
                mult = 0
                while nudge not in cone_dict["cone"]:
                    mult += 1
                    nudge += ray_sum

                if mult not in cone_points:
                    mults = free_module_element([mult for k in range(self._num_variables)])
                    higher_points = [b + mults for b in cone_points[0][0]]
                    cone_points[mult] = (higher_points, [tuple(b) for b in higher_points])

                num_integral_points = []
                for point in cone_points[mult][0]:
                    polytope_point = cone_dict["lift_matrix"]*point + off_set.lift()
                    polytope = self._create_polytope_from_matrix(polytope_point)
                    num_integral_points.append(len(polytope.integral_points()))

                # print(max_degree, cone_points[mult][1], num_integral_points)
                off_set_poly = self._R.interpolation(max_degree,
                                                     cone_points[mult][1],
                                                     num_integral_points)
                polynomials[off_set] = off_set_poly

            cone_dict["polynomials"] = polynomials

PEQP._compute_piecewise = _compute_piecewise

In [133]:
D = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
s = PEQP(D)

Finitely generated module V/W over Integer Ring with invariants (2, 2, 2)
(0, 0, 0)
(0, 0, 1)
(0, 1, 0)
(0, 1, 1)
(1, 0, 0)
(1, 0, 1)
(1, 1, 0)
(1, 1, 1)


In [135]:
s._cone_dicts

[{'scaled_rays': ((0, 0, 1, 0, 0, 1), (0, 1, 0, 0, 1, 0), (1, 0, 0, 1, 0, 0)),
  'cone': 6-d cone in 6-d lattice N,
  'quotient': Finitely generated module V/W over Integer Ring with invariants (2, 2, 2),
  'basis': ((0, 0, 1, 0, 0, 1), (0, 1, 0, 0, 1, 0), (1, 0, 0, 1, 0, 0)),
  'change_of_basis_matrix': [ 0  0  1  1  0  0]
  [ 0  1  0  0  1  0]
  [ 1  0  0  0  0  1]
  [ 0  0  1 -1  0  0]
  [ 0  1  0  0 -1  0]
  [ 1  0  0  0  0 -1],
  'change_of_basis_inverse': [   0    0  1/2    0    0  1/2]
  [   0  1/2    0    0  1/2    0]
  [ 1/2    0    0  1/2    0    0]
  [ 1/2    0    0 -1/2    0    0]
  [   0  1/2    0    0 -1/2    0]
  [   0    0  1/2    0    0 -1/2],
  'lift_matrix': [0 0 1]
  [0 1 0]
  [1 0 0]
  [0 0 1]
  [0 1 0]
  [1 0 0],
  'polynomials': {(0, 0, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 4*x1*x2 + 2*x0 + 2*x1 + 2*x2 + 1,
   (0, 0, 1): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 2*x0,
   (0, 1, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x1*x2 + 2*x1,
   (0, 1, 1): 8*x0*x1*x2 + 4*x0*x1,
   (1, 0, 0): 8*x

In [134]:
test_combinations(s, D, range(-6, 7))

all points were tested


In [136]:
E = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 1, 1]])
t = PEQP(E)

Finitely generated module V/W over Integer Ring with invariants (4)
(0)
(1)
(2)
(3)


In [138]:
t._cone_dicts

[{'scaled_rays': ((1, 1, 1, 1),),
  'cone': 4-d cone in 4-d lattice N,
  'quotient': Finitely generated module V/W over Integer Ring with invariants (4),
  'basis': ((1, 1, 1, 1),),
  'change_of_basis_matrix': [ 1  1  0  0]
  [ 1  0  1  0]
  [ 1  0  0  1]
  [ 1 -1 -1 -1],
  'change_of_basis_inverse': [ 1/4  1/4  1/4  1/4]
  [ 3/4 -1/4 -1/4 -1/4]
  [-1/4  3/4 -1/4 -1/4]
  [-1/4 -1/4  3/4 -1/4],
  'lift_matrix': [1]
  [1]
  [1]
  [1],
  'polynomials': {(0): 32/3*x^3 + 16*x^2 + 22/3*x + 1,
   (1): 32/3*x^3 + 8*x^2 + 4/3*x,
   (2): 32/3*x^3 - 2/3*x,
   (3): 32/3*x^3 - 8*x^2 + 4/3*x}}]

In [137]:
test_combinations(t, E, range(-6, 7))

all points were tested


In [140]:
ehr(E, t._rays[0])

QuasiPolynomialElement(Ring of Quasi-Polynomials over Rational Field, [[1], [22/3], [16], [32/3]])